In [1]:
"""
Request data from the Leage of Legends API and write it to a text file
"""
import requests
import time
REQUESTLIMIT10S = 10
REQUESTLIMIT10M = 500
requestCount = 0
request = 'https://euw.api.riotgames.com/api/lol/EUW/v2.5/league/challenger?type=RANKED_SOLO_5x5&api_key=RGAPI-50a14ae0-9800-41c7-8ebc-37b646e63b56'
requestCount +=1
response = requests.get(request)
responseJson = response.json()
participants = set()
for entry in responseJson['entries']:
    participants.add(entry['playerOrTeamId'])
requestCountS =0 
requestCountM =0  
matchesById = list()
for playerId in participants:  
    if(requestCountS == REQUESTLIMIT10S): #if requests reach 10 in 10 seconds, sleep for 10 seconds
        time.sleep(10)
        requestCountS = 0   
    if(requestCountM == REQUESTLIMIT10M): #if requests reach the limit of 500 in 10 minutes, sleep for 10 minutes 
        time.sleep(600)
        requestCountM = 0  
    requestCountS +=1
    requestCountM +=1           
    request = 'https://euw.api.riotgames.com/api/lol/EUW/v1.3/game/by-summoner/' + playerId + '/recent?api_key=RGAPI-50a14ae0-9800-41c7-8ebc-37b646e63b56'
    requestCount +=1
    response = requests.get(request)
    responseJson = response.json()
    if 'games' in responseJson:
        for match in responseJson['games']:
            stats = dict()
            if 'RANKED' in match['subType']:
                stats['Team1_summonerId_1'] = int(playerId)
                stats['gameId'] = match['gameId']
                stats['Team1_win'] = match['stats']['win']
                stats['Team2_win'] = not match['stats']['win']
                team1 = match['teamId']
                count_Team_1_Players = 2
                count_Team_2_Players = 1
                for player in match['fellowPlayers']:
                    if player['teamId'] is team1:
                        stats['Team1_summonerId_' + str(count_Team_1_Players)] = player['summonerId']
                        count_Team_1_Players +=1
                    else:
                        stats['Team2_summonerId_' + str(count_Team_2_Players)] = player['summonerId'] 
                        count_Team_2_Players+=1
                matchesById.append(stats)
thefile = open('matchesById.txt', 'w')
for item in matchesById:
    thefile.write("%s,\n" % item) 
thefile.close()

In [15]:
''' Conver text file to json readable format'''
file = open('matchesById.txt') 
out = '{"games" :['
for line in file:
    a=line.replace('\'', '"')
    b = a.replace('True','true')
    c = b.replace('False','false')
    out+=c
out=out[:len(out)-2] #removing the last , to avoid syntax errors
out+=']}'    
thefile = open('matchesById.json', 'w')
thefile.write(out)
thefile.close()
file.close()

In [29]:
import json
import pandas as pd
with open('matchesById.json') as data_file:
    data = json.load(data_file)
matches = pd.DataFrame()
for match in data['games']:
    stats = pd.DataFrame(match,index = data)
    matches = pd.DataFrame.append(matches,stats)
matches.to_csv("games.csv", sep=',')